In [1]:
# importing libraries

import pandas as pd
import numpy as np
import os 

In [2]:
# Reading Data 
os.chdir(r'C:\Users\HP\Downloads\concrete_data')
concrete_data = pd.read_csv('concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
# Check Data Shape
concrete_data.shape

(1030, 9)

In [4]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.

In [6]:
# Split data into predictors and target

concrete_data_columns = concrete_data.columns
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [7]:
predictors

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360
...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28


In [8]:
target

0       79.99
1       61.89
2       40.27
3       41.05
4       44.30
        ...  
1025    44.28
1026    31.18
1027    23.70
1028    32.77
1029    32.40
Name: Strength, Length: 1030, dtype: float64

In [9]:
n_cols = predictors.shape[1] # number of predictors
n_cols

8

In [10]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [11]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=42)

In [14]:
# build the model
model = regression_model()
# fit the model
epochs = 50
model.fit(X_train, y_train, epochs=epochs, verbose=1)

Epoch 1/50
23/23 [==============================] - 1s 4ms/step - loss: 13516.0801
Epoch 2/50
23/23 [==============================] - 0s 4ms/step - loss: 2305.9395
Epoch 3/50
23/23 [==============================] - 0s 4ms/step - loss: 1091.0249
Epoch 4/50
23/23 [==============================] - 0s 4ms/step - loss: 990.7644
Epoch 5/50
23/23 [==============================] - 0s 4ms/step - loss: 871.6293
Epoch 6/50
23/23 [==============================] - 0s 4ms/step - loss: 768.0619
Epoch 7/50
23/23 [==============================] - 0s 4ms/step - loss: 674.6142
Epoch 8/50
23/23 [==============================] - 0s 4ms/step - loss: 588.1871
Epoch 9/50
23/23 [==============================] - 0s 5ms/step - loss: 512.5783
Epoch 10/50
23/23 [==============================] - 0s 5ms/step - loss: 451.2828
Epoch 11/50
23/23 [==============================] - 0s 5ms/step - loss: 397.7354
Epoch 12/50
23/23 [==============================] - 0s 4ms/step - loss: 356.5443
Epoch 13/50
23/23 [==

In [16]:
# evaluate the model on the test data
loss_val = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
loss_val

10/10 [==============================] - 0s 4ms/step


119.24113464355469

In [17]:
# Compute the mean squared error between the predicted concrete strength and the actual concrete strength.
from sklearn.metrics import mean_squared_error
mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(mean, standard_deviation)

119.24114890146001 0.0


In [18]:
total_mean_squared_errors = 50
epochs = 50
mean_squared_errors = []
for i in range(0, total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("Below is the mean and standard deviation of " +str(total_mean_squared_errors) + " mean squared errors without normalized data. Total number of epochs for each training is: " +str(epochs) + "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

MSE 1: 84.23403930664062
10/10 [==============================] - 0s 2ms/step
MSE 2: 116.73729705810547
10/10 [==============================] - 0s 2ms/step
MSE 3: 84.02308654785156
10/10 [==============================] - 0s 2ms/step
MSE 4: 88.3523941040039
10/10 [==============================] - 0s 1ms/step
MSE 5: 74.02823638916016
10/10 [==============================] - 0s 2ms/step
MSE 6: 65.25355529785156
10/10 [==============================] - 0s 2ms/step
MSE 7: 74.57231140136719
10/10 [==============================] - 0s 2ms/step
MSE 8: 51.271366119384766
10/10 [==============================] - 0s 2ms/step
MSE 9: 56.257591247558594
10/10 [==============================] - 0s 3ms/step
MSE 10: 58.79990768432617
10/10 [==============================] - 0s 2ms/step
MSE 11: 47.94743728637695
10/10 [==============================] - 0s 3ms/step
MSE 12: 42.33832550048828
10/10 [==============================] - 0s 2ms/step
MSE 13: 54.75746536254883
10/10 [==========================